In [1]:
import os
if os.path.split(os.getcwd())[1] != "MRB_II":
    %cd ../../

D:\Documents\A_DIGIPEN\PersonalSVN\Fall22SVN\CSP400\MRB_II


In [2]:
# Global
import math
from datetime import datetime as dt

import numpy as np
import pandas as pd

import nltk

from collections import Counter

# Local
from yt_utils.yt_categories import YouTubeCategories

In [5]:
df = pd.read_feather("https://squeemos.pythonanywhere.com/static/yt_categories.feather")

In [21]:
categories = YouTubeCategories("https://squeemos.pythonanywhere.com/static/video_categories.json")
categories

In [14]:
nltk.download('stopwords')
STOPWORDS = nltk.corpus.stopwords.words("english")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\eric\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Base

In [16]:
def __tokenize_tags(tags):
    out = " ".join(tags).lower().split()
    out = [w for w in out if w not in STOPWORDS]
    return out

In [17]:
# Remove features containing the following strings
drops = (
    "localizations", "liveStreamingDetails", "recordingDetails",
    "regionRestriction", "ytRating", "thumbnails", "defaultLanguage",
)
tag_df = df.loc[:, [col for col in df.columns if not any(d in col for d in drops)]]

# Get only last month of data with tokenized/lowered/stopword-free tags
tag_df = tag_df.set_index("queryTime").last("30D").reset_index()
tag_df = tag_df.dropna(subset="snippet.tags")
tag_df["tags"] = tag_df["snippet.tags"].apply(__tokenize_tags)

In [29]:
tag_df

,queryTime,kind,etag,id,snippet.publishedAt,snippet.channelId,snippet.title,snippet.description,snippet.channelTitle,snippet.categoryId,...,status.madeForKids,statistics.viewCount,statistics.likeCount,statistics.favoriteCount,statistics.commentCount,player.embedHtml,topicDetails.topicCategories,snippet.tags,snippet.defaultAudioLanguage,tags
1,2022-10-13 03:00:47+00:00,youtube#video,y9OsgzQdx0AFBXSyUZZU6ECoRW4,UIGNMccGh8w,2022-10-02 18:20:31+00:00,UCQz2x0BlBM9MVaUW_kYPosg,مشهد سينمائي من فيلم | roman j israel esq,#سينمائيونHD #أقتباسات_افلام #أقتباسات_سينمائي...,𝐑𝐢𝐜𝐤🎭,24,...,False,15589366,1036675.0,0,23024.0,"<iframe width=""480"" height=""270"" src=""//www.yo...","[https://en.wikipedia.org/wiki/Entertainment, ...","[اقتباسات افلام, اقتباسات واقوال, اقتباسات أفل...",ar,"[اقتباسات, افلام, اقتباسات, واقوال, اقتباسات, ..."
3,2022-10-13 03:00:47+00:00,youtube#video,MHIIeS45a6qMKEDn2-pFz5nTLkc,SS7HXxy3_2c,2022-10-09 05:38:34+00:00,UCqFzWxSCi39LnW1JKFR3efg,Try Guys - SNL,A CNN broadcast is interrupted by breaking new...,Saturday Night Live,23,...,False,2069780,31249.0,0,18810.0,"<iframe width=""480"" height=""270"" src=""//www.yo...","[https://en.wikipedia.org/wiki/Entertainment, ...","[SNL10082022, snl, saturday night live, snl 48...",en,"[snl10082022, snl, saturday, night, live, snl,..."
4,2022-10-13 03:00:47+00:00,youtube#video,pvxaPRxR_IfrhyuT32U7XIBBjBs,oibJn6Ct1tg,2022-10-03 17:21:17+00:00,UC7bouvhSTd2RQwYOi7zq0hQ,Interrupted in San Diego #shorts #comedy #funny,i appreciate the gift but not the poor crowd e...,Stavros Halkias,23,...,False,8233551,659746.0,0,2802.0,"<iframe width=""480"" height=""270"" src=""//www.yo...","[https://en.wikipedia.org/wiki/Entertainment, ...","[stand up comedy, crowd work comedy, crowdwork...",en,"[stand, comedy, crowd, work, comedy, crowdwork..."
6,2022-10-13 03:00:47+00:00,youtube#video,_H9hxMLBhfIwBuWlSPIFgDHXak4,c2s1ASeKDhc,2022-10-06 21:42:10+00:00,UC0L0KUZRoSh0Xt7UL5xFnsA,THE SUPER MARIO BROS. MOVIE | OFFICIAL TEASER ...,The official teaser trailer for The Super Mari...,Universal Pictures Australia,24,...,False,3511043,90810.0,0,8360.0,"<iframe width=""480"" height=""270"" src=""//www.yo...","[https://en.wikipedia.org/wiki/Entertainment, ...","[Universal, Illumination Entertainment, Ninten...",en,"[universal, illumination, entertainment, ninte..."
7,2022-10-13 03:00:47+00:00,youtube#video,4XPQq7vucOenCLBJOAmoMSKCIDA,_xQ27AsF4_k,2022-10-12 02:29:49+00:00,UCwWhs_6x42TyRM4Wstoq8HA,Kanye West Makes Anti-Semitic Comments & Draym...,Kanye West is blocked from Twitter after anti-...,The Daily Show with Trevor Noah,23,...,False,1549624,47998.0,0,3246.0,"<iframe width=""480"" height=""270"" src=""//www.yo...","[https://en.wikipedia.org/wiki/Entertainment, ...","[the daily show, trevor noah, daily show with ...",en-US,"[daily, show, trevor, noah, daily, show, trevo..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239288,2022-10-30 06:00:53+00:00,youtube#video,QWtI1iBaOV8BVfOalLKhdu8dn9Q,Ix28jjDCEh4,2022-10-26 00:39:24+00:00,UC_Vl1oLTGjWYJLmbTpaqorQ,"How Ned Fulmer Ruined the Try Guys Reputation,...",Thanks to Established Titles for sponsoring th...,Spill,24,...,False,378347,14522.0,0,816.0,"<iframe width=""480"" height=""270"" src=""//www.yo...",[https://en.wikipedia.org/wiki/Lifestyle_(soci...,"[ned & ariel, try guys, keith, ned fulmer, ned...",en,"[ned, &, ariel, try, guys, keith, ned, fulmer,..."
239289,2022-10-30 06:00:53+00:00,youtube#video,m9uoydFaMhG1JJv4GmOtLtIoQqU,t0rz5WfYg4w,2022-10-14 16:37:07+00:00,UCoVvd6bZ6LSKv-ePiow1evg,Bit of blob to cover your sins. #shorts,,CJR ELECTRICAL,26,...,False,15560403,173933.0,0,1140.0,"<iframe width=""480"" height=""270"" src=""//www.yo...",[https://en.wikipedia.org/wiki/Lifestyle_(soci...,[Cjrelectrical],en,[cjrelectrical]
239290,2022-10-30 06:00:53+00:00,youtube#video,t5LktK5Ss9waKilTzijLipz9O4o,3tRsaJ2AuU0,2022-10-23 17:00:29+00:00,UC5Qbo0AR3CwpmEq751BIy0g,10 NEW Costco Deals You NEED To Buy in October...,So many

### Hist

In [26]:
# Get only latest videos
cat_tags = tag_df.drop_duplicates(subset="id", keep="last", ignore_index=True)

# Get tags for each category
cat_tags = cat_tags.groupby("snippet.categoryId")["tags"].sum()

cat_tags

snippet.categoryId
1     [hbo, hbo, max, hbo, max, movie, hbo, max, ser...
2     [road, recovery, rescue, matts, road, recovery...
10    [boywithuke, sick, mercury, records/republic, ...
15    [animal, video, animals, dodo, animal, rescue,...
17    [chelsea, milan, milan, vs., chelsea, televisa...
19    [toya, johnson, weight, ti, &, tiny, friends, ...
20    [simulation, games, graystillplays, simulator,...
22    [daily, bumps, daily, bumps, baby, welcome, br...
23    [jimmy, jimmy, kimmel, jimmy, kimmel, live, la...
24    [piers, morgan, piers, morgan, andrew, tate, a...
25    [tulsi, gabbard, tulsi, gabbard, fox, news, ga...
26    [girls, night, daily, vlog, zara, perfume, atl...
27    [top, 10, central, karen, karen, vs, police, k...
28    [shorts, tech, myths, busted, tips, pcbs2, pc,...
29    [behold, israel, amir, tsarfati, bible, bible,...
Name: tags, dtype: object

In [27]:
# Create column for category id and category name
cat_tags = pd.DataFrame(cat_tags).reset_index()
cat_tags["category"] = cat_tags["snippet.categoryId"].apply(lambda x: categories.id_to_title[x])
cat_tags

,snippet.categoryId,tags,category
0,1,"[hbo, hbo, max, hbo, max, movie, hbo, max, ser...",Film & Animation
1,2,"[road, recovery, rescue, matts, road, recovery...",Autos & Vehicles
2,10,"[boywithuke, sick, mercury, records/republic, ...",Music
3,15,"[animal, video, animals, dodo, animal, rescue,...",Pets & Animals
4,17,"[chelsea, milan, milan, vs., chelsea, televisa...",Sports
5,19,"[toya, johnson, weight, ti, &, tiny, friends, ...",Travel & Events
6,20,"[simulation, games, graystillplays, simulator,...",Gaming
7,22,"[daily, bumps, daily, bumps, baby, welcome, br...",People & Blogs
8,23,"[jimmy, jimmy, kimmel, jimmy, kimmel, live, la...",Comedy
9,24,"[piers, morgan, piers, morgan, andrew, tate, a...",Entertainment


In [28]:
cat_tags["tags"] = cat_tags["tags"].apply(Counter)
cat_tags["tags"] = cat_tags["tags"].apply(lambda x: dict(x.most_common(10)))
cat_tags

,snippet.categoryId,tags,category
0,1,"{'movie': 38, 'leroy': 35, '2022': 26, 'vs': 2...",Film & Animation
1,2,"{'road': 101, 'recovery': 78, 'lz': 39, 'cars'...",Autos & Vehicles
2,10,"{'grizzley': 66, 'video': 65, 'tee': 60, 'offi...",Music
3,15,"{'animals': 52, 'dodo': 37, 'dog': 35, 'funny'...",Pets & Animals
4,17,"{'first': 200, 'highlights': 189, 'espn': 183,...",Sports
5,19,"{'haulover': 16, 'boat': 15, 'vlog': 14, 'boat...",Travel & Events
6,20,"{'minecraft': 866, 'roblox': 730, 'gta': 563, ...",Gaming
7,22,"{'vlog': 149, 'nichlmao': 100, 'life': 76, 'fa...",People & Blogs
8,23,"{'comedy': 178, 'funny': 140, 'stand': 79, 'sh...",Comedy
9,24,"{'minecraft': 437, 'funny': 356, 'shorts': 209...",Entertainment
